### torchtext 라이브러리로 텍스트 분류 <hr>
- 1단계 - 데이터 전처리 : 숫자형식으로 변환하는 것 까지
- 2단계 - 모델 구현

[1-1] 데이터 준비 => 내장 데이터셋 활용
- AG_NEWS 데이터셋 반복자 : 라벨(label) + 문장의 튜플(tuple) 형태 

In [1]:
import torch
from torchtext.datasets import AG_NEWS

# DataPipe 타입 >>> iterator 타입 형변환
train_iter = iter(AG_NEWS(split='train'))

In [2]:
# !pip install dataset/

In [3]:
# 데이터 확인 => (label, text), label 1 ~ 4
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

[2] 데이터 처리 파이프라인 준비
- 어휘집합(Vocab) 생성, 단어벡터(word vector), 토크나이저(tokenizer)
- 가공되지 않은 텍스트 문자열에 대한 데이터 처리 빌딩 블록
- 일반적인 NLP 데이터 처리
    * 첫번째 단계 : 가공되지 않은 학습 데이터셋으로 어휘집 생성
        - 토큰 목록 또는 반복자 받는 내장 핵토리 함수(factory function) : ``build_vocab_from_iterator``
        - 사용자는 어휘집에 추가할 특수 기호(special symbol) 전달 가능
    * 두번째 단계 : 어휘집 사용하여 문장 토큰화
    * 세번째 단계 : 문장 토큰화 결과를 문자열 형태로 변환


In [1]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# 토커나이즈 생성
tokenizer = get_tokenizer('basic_english')

# 뉴스 학습 데이터 추출
train_iter = AG_NEWS(split='train')

NameError: name 'AG_NEWS' is not defined

In [5]:
## => 토큰 생성 제너레이터 함수 : 데이터 추출하여 토큰화
def yield_tokens(data_iter):
    # 라벨, 텍스트 -> 텍스트 토큰화( 문장마다 들어가는 단어를 쪼갠 것 )
    for _, txt in data_iter:
        yield tokenizer(txt)

In [1]:
## 단어사전 생성
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<UNK>"]) # min_freq = 기본값 : 1

## <UNK> 인덱스 0으로 설정
vocab.set_default_index(vocab["<UNK>"])

NameError: name 'build_vocab_from_iterator' is not defined

In [8]:
vocab(['<UNK>','here','is',' an','example'])
# 단어사전에다가 단어를 넣었을때 인덱스 반환

[0, 475, 21, 0, 5297]

In [9]:
# 텍스트 >> 정수 인코딩
text_pipeline = lambda x:vocab(tokenizer(x))

# 레이블 >> 정수 인코딩( 0 ~ 3 )
label_pipeline = lambda x: int(x) - 1

In [10]:
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 배치 크기만큼 데이터셋 반환 함수
def collate_batch(batch):
    '''
    batch : (label, text)
    '''
    # 배치 크기만큼의 라벨 텍스트 오프셋 값 저장 변수
    label_list, text_list, offsets = [], [], [0] # offset : 첫번째 데이터의 갯수
    
    # 뉴스기사 1개씩 라벨 추출해서 저장
    for (_label, _text) in batch:
        
        # 라벨 인코딩 후 저장
        label_list.append(label_pipeline(_label))

        # 텍스트 인코딩 후 저장
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)

        # 텍스트 offset 즉, 텍스트 크기/길이 저장
        offsets.append(processed_text.size(0))
 
    # 텐서화 진행
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0) # 누적값 -> 전체 갯수
    text_list = torch.cat(text_list)
    
    return label_list.to(device), text_list.to(device), offsets.to(device)


# 이렇게 할땐 패딩 => embeddingbag 사용 - 길이 다 다른 문장 알아서 정리

In [11]:
train_iter = AG_NEWS(split = 'train')
dataloader = DataLoader(train_iter,
                        batch_size = 8, # 기사를 8개 뽑아온다는 뜻 -> 각 기사마다 길이가 모두 다름
                        shuffle = False,
                        collate_fn = collate_batch)

In [12]:
# 분류 클래스 수와 단어사전 개수
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)

print(f'num_class : {num_class}   vocab_size : {vocab_size}')

num_class : 4   vocab_size : 95811
